In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from processingQst import *
from similarityFctsHPC import *
from tqdm import tqdm
import pandas as pd
import multiprocessing

In [ ]:
# namefile = "train"
namefile = "dev"
df = pd.read_csv(f"./input/csv/{namefile}.csv",delimiter=";")


Qst = df["question"]
id  = df["id"]
nbrRepAll = df["nbr_correct_answers"]
RepListIndxAll = df["correct_answers"]
nbrRep = 5
print(len(Qst))
data_list = []
data_listAux = []
RepListIndx = 'abcde'

In [ ]:
with open('output/corpusMS.json', 'r') as f:
    corpusA = json.load(f)
with open('output/corpusMerck.json', 'r') as fs:
    corpusB = json.load(fs)

In [ ]:

# for cpt in range(len(Qst)):
for cpt in tqdm(range(len(Qst))):
    # if (cpt > 100):
    #     break
    if (cpt>=0): 
        qst = Qst[cpt]
        # print(qst)
        nwQst = ''
        listMed = []
        keywordQ = []
        keywordQP = []
        ListMedAns = ''
        Spword = False
        data_dict = {}

        # Détécter la négation 
        if(isRequestWrongAns(qst)):
            WrongQst = True 
            # print(WrongQst)
        else: 
            WrongQst = False 
            # print(WrongQst)
        
        # supprimer les parenthèses 
        qst = del_betParenthese(qst)

        # fragementation des questions selon les ","
        fragQst = splitQst(qst)

        for icpt in range(len(fragQst)):
            if not (fragQst[icpt]==''):
                nwQst,Spword,listMed = recoverMedFrag(fragQst,icpt,nwQst,Spword)
                keywordQ.extend(listMed)
        nwQst = nwQst.lstrip()
        nwQstp = ''
        var = ''
        varcpt = 0
        for j in range(5):
            keywordQP = []
            Ans = df[f"answers.{RepListIndx[j]}"][cpt] 
            Ans = Ans.lstrip()
            Ans = re.sub(pattern, add_spaces, Ans)
            # print(Ans)
            keywordQP.extend(RecoverListMed(Ans))
            if Spword:
                nwQstp = replaceAtFront(Ans) + nwQst + '.'
            else:
                nwQstp = nwQst + ' '+ Ans.lower() + '.' 
            # Appliquer la négation 
            if WrongQst == True:
                nwQstp = appliquer_negation(nwQstp.lower())
            
            # print(keywordQ)
            # print(keywordQP)
            taux_sim = getRepDB(keywordQ,keywordQP,corpusA,nwQstp)
            # taux_sim = getRepDB_parallel(keywordQ,keywordQP, corpusA,nwQstp)
            # print(taux_sim)
            # print(taux_simB)
            if taux_sim>0.9:
                if var !="":
                    var = var + '|'
                var = var + RepListIndx[j]
                varcpt +=1
            else:
                # taux_simB = getRepDB(keywordQ,keywordQP,corpusB,nwQstp)
                taux_simB = getRepDB_parallel(keywordQ,keywordQP,corpusB,nwQstp)
                if taux_simB>0.68:
                    if var !="":
                        var = var + '|'
                    var = var + RepListIndx[j]
                    varcpt +=1

            
        # créer un dataframe pour les données actuelles
        data_dict = {'id': id[cpt], 'correct_answers': var}
        data_dictAux = {'id': id[cpt], 'nbr_correct_answers': varcpt}
        data_list.append(data_dict)
        data_listAux.append(data_dictAux)



In [ ]:
# writeJson("output/Qst_dev.json",data_list)
# ---
df = pd.DataFrame(data_list)
df.to_csv(f"./output/tachePrincipale{namefile}.csv", sep=';', index=False)
# ---
dfs = pd.DataFrame(data_listAux)
dfs.to_csv(f"./output/tacheAnnexe{namefile}.csv", sep=';', index=False)